I guess this is an example of an autoencoder that I can use?

In [ ]:
# autoencoder_mnist_dnn_tqdm.py
from torch import optim, nn, no_grad
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
from tqdm import tqdm

transform = transforms.ToTensor()
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

class AutoEncoder(nn.Module):
    def __init__(self, bottleneck_dim=4):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 64)
        self.fc2 = nn.Linear(64, bottleneck_dim)

        self.fc3 = nn.Linear(bottleneck_dim, 64)
        self.fc4 = nn.Linear(64, 28 * 28)

        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))

        z = self.fc2(x)

        x = self.relu(self.fc3(z))
        x = self.sigmoid(self.fc4(x))

        return x.view(-1, 1, 28, 28)

device = "cpu"
model = AutoEncoder(bottleneck_dim=3).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 5

for epoch in range(1, epochs + 1):
    model.train()

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}")
    for x, _ in pbar:
        x = x.to(device)

        pred = model(x)
        loss = criterion(pred, x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_postfix(loss=loss.item())

model.eval()
with no_grad():
    x, _ = next(iter(test_loader))
    x = x.to(device)
    pred = model(x)

save_image(make_grid(x[:8]), "original.png")
save_image(make_grid(pred[:8]), "recon.png")